In [2]:
import pandas as pd

from src.paths import PREPROCESSED_DATA_DIR

In [36]:
df_weather = pd.read_pickle(PREPROCESSED_DATA_DIR / 'clean_weather.pkl')
print(f'Columns: {df_weather.columns.to_list()}')

Columns: ['Date', 'Tmax_1', 'Tmin_1', 'Tavg_1', 'DewPoint_1', 'WetBulb_1', 'PrecipTotal_1', 'Tmax_2', 'Tmin_2', 'Tavg_2', 'DewPoint_2', 'WetBulb_2', 'PrecipTotal_2']


In [37]:
df_weather.set_index('Date', inplace=True)

In [47]:
def flatten_index(df):
    df.columns = [c[0]+'_'+c[1] for c in df.columns.to_flat_index()]

def add_lag_window(df, lag, window):
    df.columns = ['_'.join([c,str(lag),str(window)]) for c in df.columns]
    
def aggregate_columns(df):
    df_agg = pd.DataFrame(index=df.index)
    for lag in range(1, 30):
        for window in range(1,30):
            df_one = df.shift(lag).rolling(window).agg(['sum', 'mean'])
            flatten_index(df_one)
            add_lag_window(df_one, lag, window)
            df_agg = pd.concat([df_agg, df_one], axis=1).dropna()
    return df_agg


df_agg = aggregate_columns(df_weather)

In [49]:
df_agg.to_pickle(PREPROCESSED_DATA_DIR / 'aggregated_weather_all.pkl')